In [1]:
from pyspark.sql import SparkSession
import time

# 1. Создаем сессию с явным указанием ресурсов для наглядности
spark = (SparkSession.builder
         .appName("Module2_Architecture_Lab")
         .master("local[*]") # Используем все ядра локально
         .config("spark.executor.memory", "1g")
         .getOrCreate())

# Ссылка на UI
print(f"Go to Spark UI: http://localhost:4040")

Go to Spark UI: http://localhost:4040


In [2]:
# Создаем DataFrame (это Transformation - ленивая операция)
# Генератор данных: создаем список от 0 до 1 млн
data = spark.range(0, 1000000)

# Пока что Job не создан! Проверьте вкладку "Jobs" в UI - там пусто.

In [3]:
# Вызываем Action
count = data.count()

print(f"Количество строк: {count}")
# ТЕПЕРЬ обновите вкладку "Jobs" в Spark UI. Появился Job Id 0.

Количество строк: 1000000


In [4]:
# Repartition вызывает Shuffle (перераспределение данных)
# Это Transformation
repartitioned_data = data.repartition(4)

# Action - чтобы запустить выполнение
repartitioned_data.count()

# Идите в Spark UI -> Jobs.
# Кликните на новый Job (Description).
# Вы увидите DAG Visualization: там будет 2 квадрата (Stage), соединенных стрелкой Exchange.
# Exchange = Shuffle.

1000000

In [6]:
# Не забываем закрывать
spark.stop()

#### tmp

In [1]:
from pyspark.sql import SparkSession

# 1. Создаем сессию с явным указанием ресурсов для наглядности
spark = (SparkSession.builder
         .appName("Module2_Architecture_Lab")
         .master("local[*]") # Используем все ядра локально
         .getOrCreate())

In [2]:
# Генерируем 10 млн чисел и разбиваем на 4 раздела (Tasks)
# Все разделы будут одинакового размера (по 2.5 млн)
df_balanced = spark.range(0, 10000000).repartition(4)

# Action запускает Job
df_balanced.count()

# ЗАДАНИЕ:
# 1. Зайдите в Stages. Найдите последний Stage.
# 2. Посмотрите Summary Metrics.
# 3. Ожидание: Duration у Min и Max почти одинаковое.

10000000

In [4]:
from pyspark.sql import functions as F

# Создаем данные, где 99% ключей = 1, а остальные случайные
# Это классический случай "товара-локомотива" в продажах
df_skewed = spark.range(0, 1000000).withColumn("key", 
    F.when(F.col("id") < 990000, 1).otherwise(F.col("id"))
)

# Вызываем Shuffle (groupBy вызывает перемешивание данных)
# Данные полетят на экзекьюторы в зависимости от ключа.
# Все записи с ключом 1 (990 тыс. штук) полетят на ОДИН Task.
df_skewed.groupBy("key").count().collect()

# ЗАДАНИЕ:
# 1. Зайдите в Stages. Найдите Stage с "Aggregate".
# 2. Посмотрите Summary Metrics.
# 3. Ожидание: Max Duration будет в разы больше Median.
# 4. В Event Timeline одна полоска будет очень длинной, остальные короткие.